In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from fastonn import OpNetwork,utils,OpTier,OpBlock,SelfONN, Trainer
from fastonn.osl import *

#define the operation set library:
OPLIB = getOPLIB(
    NODAL = [mul, cubic, sine, expp, sinh, chirp],
    POOL = [sum],
    ACTIVATION =[tanh] 
)


print('type', type(OPLIB), str(OPLIB))

#configure ONN network:
model = OpNetwork(
    in_channels=1,
    tier_sizes=[12,32,1], #number of neurons in hidden and input layer
    kernel_sizes=[21,7,3],
    operators=[
        [1], #operators for 1st hidden layer
        [6], #operators for 2nd hidden layer
        [3] #output layer
    ],
    sampling_factors= [2, -2, 1], #scaling factors for each layer
    OPLIB = OPLIB, #assign operator library
)

print(model)


#import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')




type <class 'list'> [{'nodal': <function mul at 0x000002CB4DCC9BD0>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}, {'nodal': <function cubic at 0x000002CB4DCC9CF0>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}, {'nodal': <function sine at 0x000002CB4DCC9D80>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}, {'nodal': <function expp at 0x000002CB4DCC9E10>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}, {'nodal': <function sinh at 0x000002CB4DCC9EA0>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}, {'nodal': <function chirp at 0x000002CB4DCCA200>, 'pool': <built-in function sum>, 'act': <function tanh at 0x000002CB4DCCA440>}]
OpNetwork(
  (oper): Sequential(
    (0): OpTier(
      (oper): ModuleList(
        (0-11): 12 x OpBlock()
      )
    )
    (1): OpTier(
      (oper): ModuleList(
        (0): OpBlock()
      )
    )
   